In [20]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

melb_df = melb_data.copy()
melb_df.head()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
#display(total_rooms)

melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
#display(melb_df['MeanRoomsSquare'])

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])
#display(melb_df['Date'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])
years_sold = melb_df['Date'].dt.year
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
#display(delta_days)
#display(delta_days.dt.days)
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#print(weekend_count)

#print(melb_df['Address'].nunique())
#print(melb_df['Address'].loc[177])
#print(melb_df['Address'].loc[1812])
#print(melb_df['Address'].loc[9001])


# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
#print(street_types.nunique())
#display(street_types.value_counts())
popular_stypes =street_types.value_counts().nlargest(10).index
#print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])
#print(melb_df['StreetType'].nunique())
melb_df = melb_df.drop('Address', axis=1)

#print(melb_df['WeekdaySale'])

#for weekday in melb_df['WeekdaySale']
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
#print(melb_df['Weekend'])
#print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 2))


popular_agents = melb_df['SellerG'].value_counts().nlargest(49).index
#print(popular_stypes)

melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_agents else 'other')
#display(melb_df['StreetType'])
#print(melb_df['StreetType'].nunique())
#print(melb_df['SellerG'])

a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
#print(round(a/b, 1))


# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений, тип)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)

#Тип данных Category

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

#print(melb_df['Regionname'].cat.categories)
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
#display(melb_df['Type'])
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
#display(new_houses_types)

#melb_df.info()
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

data_ufo

In [6]:
import pandas as pd

data_ufo = pd.read_csv('data/data_ufo.csv', sep=',')
data_ufo['Time'] = pd.to_datetime(data_ufo['Time'])
#display(data_ufo)
data_ufo_year = data_ufo['Time'].dt.year
#print('max:', data_ufo_year.mode()[0])
data_ufo['Date'] = data_ufo['Time'].dt.date
print(data_ufo[data_ufo['State']=='NV']['Date'].diff().dt.days.mean())


68.92932862190813


In [7]:
import pandas as pd

print(melb_df['Address'].nunique())


KeyError: 'Address'